# Building Intelligent Agents with LangChain: A Practical Guide

LangChain has revolutionized the way we build AI applications by providing a robust framework for creating specialized agents that can perform complex tasks autonomously. This notebook demonstrates the practical implementation of five distinct agents, each designed to showcase different capabilities of the LangChain framework.

## Overview of Agents

**Basic Prompt-Tuned Agent**
We begin with a fundamental agent that demonstrates how system prompt engineering can significantly enhance an agent's performance. This serves as an excellent introduction to agent architecture and behavior customization.

**Search-Enhanced Agent**
Building upon the basics, we implement a search-capable agent using TavilySearch integration. This agent showcases how to effectively combine language models with real-time web search capabilities to provide up-to-date and accurate information.

**SQL Database Agent**
The SQL Database Agent illustrates the power of combining natural language processing with database operations. This agent can interpret natural language queries and convert them into SQL commands, making database interactions more accessible to non-technical users.

**Arxiv Paper Summarizer**
This specialized agent demonstrates how to create task-specific agents by integrating with the Arxiv API. It can fetch, process, and summarize academic papers, making research more accessible and digestible.

**Custom Plot Agent**
The final agent showcases data visualization capabilities by creating custom plots based on natural language descriptions. This implementation highlights how to combine language models with visualization libraries for intuitive data representation.

Through these implementations, we explore key concepts in agent development, including:
- Tool integration and customization
- Prompt engineering and system message design
- API interactions and data processing
- Visualization and output formatting

This notebook serves as both a practical guide and a reference for building specialized AI agents using LangChain, suitable for developers looking to create their own custom AI solutions.

## Setup APIs

Before you get started, make sure you keep the API keys ready for the following tools:

1. [groq cloud](https://console.groq.com)
2. [Tavily Search](https://tavily.com)


In [ ]:
import os

TAVILY_API_KEY='YOUR_API_KEY'
GROQ_API_KEY='YOUR_API_KEY'

os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

## Setting up the llm

We'll be using the most powerful `llama3-70b` model hosted by groq for free.

In [ ]:
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent

# Make sure you setup your ChatGroq api key first
llm = ChatGroq(model_name="llama3-70b-8192")
response = llm.invoke('Hey, tell me a joke!')

print(response.content)

## Prompt-Tuned Agent

* We will build an agent that can only answer questions about crypto.

* For this, we are only going to tune the system prompt of our llm.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

# Defining our prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You only answer to crypto-related questions, and for all other questions: you say 'I cannot help you with that!'"),
    ("placeholder", "{messages}"),
    ("user", "Remember, always be polite!") ])

# Creating an agent chain using pipe '|'
agent = prompt | llm

response = agent.invoke({"messages": ['Hey, tell me a joke']})

# Remember that our agent will only answer questions relating to crypto.
print(response.content)


In [ ]:
# Lets ask something that's related to crypto
response = agent.invoke({"messages": [HumanMessage(content='Hey, what is btc?')]})
print(response.content)

## Search-Enhanced Agent

Lets integrate `TavilySearchTool` which will be helpful for our agent to provide responses using the internet.

For this purpose, we are going to turn to `create_react_agent` to build our agent.

Make sure you keep the free [TavilySearch](https://tavily.com/) API key ready.



In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Ready the search tool
search = TavilySearchResults(max_results=2)
tools = [search]

In [ ]:
# Lets slightly modify our system prompt to ask the llm to prompt the user with follow up questions

prompt = ChatPromptTemplate.from_messages([
    ("system", "You only answer to crypto-related questions, and for all other questions: you say 'I cannot help you with that!'. Also prompt the user with example follow up questions."),
    ("placeholder", "{messages}"),
    ("user", "Remember, always be polite!") ])

In [ ]:

# Here we initilialize our llm along with tools and prompt template
agent_executor = create_react_agent(llm, tools, prompt=prompt)

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content='what is btc?!')]}
):
    if 'agent' in chunk:
      print(chunk['agent']['messages'][0].content)
      print("----")

In [ ]:
# Asking a follow up question (which our agent fails to answer accurately)
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content='Yes, am interested to know how it works')]}
):
    if 'agent' in chunk:
      print(chunk['agent']['messages'][0].content)
      print("----")

### Memory Problem

The only problem with this approach is that the agent do not keep track of the flow. Hence, if you ask any follow up question, it doesn't remember and not answer you well.

### Adding memory to our agent

It is essential to integrate memory into your LLMs to maintain persistence.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
config = {"configurable": {"thread_id": "abc123"}}

# Define your agent and add this memory under 'checkpointer' parameter
agent_executor = create_react_agent(llm, tools, checkpointer=memory, prompt=prompt)

In [ ]:
# Run the agent

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content='What is btc?!')]}, config
):
    if 'agent' in chunk:
      print(chunk['agent']['messages'][0].content)
      print("----")

# Asking a follow up question.
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content='Yes, I want to know how it works.')]}, config
):
    if 'agent' in chunk:
      print(chunk['agent']['messages'][0].content)
      print("----")

In [ ]:
# In case you're interested to see how the chunk looks like under the hood
print(chunk)

## SQL Database Agent




### Create sample database

We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.

In [ ]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [ ]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [ ]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [ ]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [ ]:
for obs in observations:
    insert_obs(obs)

### Building a SQL agent using `create_sql_agent`

In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    max_iterations=3
)

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [ ]:
result = count_tokens(
    agent_executor,
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)

## Arxiv Paper Summarizer

This is how you can use the ArXiv tool to retrieve paper summaries on various topics:

In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["arxiv"])

In [ ]:
# Call arXiv
print(tools[0].invoke("The attention is all you need")[:250])

Well, it's not the classic paper we're interested to learn about right. We could plug this tool to our agent and see if we can learn more about the attention.

In [ ]:
agent = create_react_agent(llm, tools)

In [ ]:
for event in agent.stream({"messages": [HumanMessage(content='what is attention in machine learning?')]}):
      if event.get('tools'):
        tool_call = event['tools']
      print(event)
      print("----")

So as you can see our agent calls the tool and extracts necessary information and provides it to our model.

In [ ]:
# Tool call - You can find what info has been requested by the agent to the tool
print(tool_call['messages'][0].content)

In [ ]:
print(event['agent']['messages'][0].content)

## Custom plot agent

Lets build a custom agent that draws a line chart based on a given data.

**Note:**
* This agent may take atleast upto 2-3mins to run.
* Re-run the cell in case it doesn't plot anything.

In [ ]:
from langchain.agents import Tool
import matplotlib.pyplot as plt

def plot_data(data: str):
    """Plots the given data."""
    x, y = eval(data)
    plt.plot(x, y)
    plt.savefig("plot.png")
    return "Plot saved as plot.png"

matplotlib_tool = Tool(
    name="plot_data",
    func=plot_data,
    description="Plots data using Matplotlib. Provide data as a list of tuples, e.g. '[(1, 2), (2, 4), (3, 6)]'"
)

In [ ]:
from langchain.agents import initialize_agent, AgentType

plot_agent = create_react_agent(llm, [matplotlib_tool])


In [ ]:
for event in plot_agent.stream({"messages":["[(1, 2), (2, 4), (3, 6)]"]}):

      print(event)
      print("----")